In [1]:
file_path = '../files/midi/Montgomery,_Wes_-_West_Coast_Blues_gp3/test.mid'
info_path = '../files/audio/Montgomery,_Wes_-_West_Coast_Blues_gp3/info.json'
import mido
import json
import numpy as np

In [2]:
song = mido.MidiFile(file_path)
with open(info_path, 'r') as fp:
        info_dictionary = json.load(fp)

In [3]:
print(info_dictionary)

{'track_0': {'type': 'guitar'}, 'track_1': {'type': 'guitar'}, 'track_2': {'type': 'guitar'}, 'track_3': {'type': 'bass'}, 'track_4': {'type': 'guitar'}, 'track_5': {'type': 'percussion'}}


In [25]:
def get_song_tempo(s):
    # returns a list of the tempi found
    tempi_musec = []
    # get tempo
    for track in s.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                tempi_musec.append( msg.tempo )
    # if no tempo found, get default mido tempo
    if len(tempi_musec) == 0:
        # default mido tempo, in microseconds per tick
        tempi_musec = [500000]
    return tempi_musec
# end get_song_tempo

480
150
0.39999999999999997


In [ ]:
tempi_musec = get_song_tempo(song)
# we don't want multiple tempi
if len( tempo_musec ) > 1:
    print('ERROR: multiple tempi, skipping file')
else:
    tempo_musec = tempi_musec[0]
    # for sanity check
    tempo_bpm = mido.bpm2tempo(tempo_musec)
    print(tempo_bpm)

In [36]:
# get note ons/offs
def get_noteOnOffs_from_midi_track(s, info_dictionary, track_name='bass'):
    # s: mido song
    # info_dictionary
    # track_name: string of track name
    for i_track, track in enumerate(s.tracks):
        tmp_track_name = info_dictionary['track_' + str(i_track)]['type']
        if tmp_track_name == track_name:
            note_ons = -1*np.ones(127).astype(int)
            note_ons_offs = []
            note_on_num = 0
            cummulative_time = 0
            # count how many notes in the bass channel
            for i_message, message in enumerate(track):
                cummulative_time += message.time
                if message.type == 'note_on':
                    if message.velocity > 0:
                        # keep the index that triggered the note on
                        note_ons[message.note] = note_on_num
                        tmp_note_obj = {
                            'on_idx': i_message,
                            'off_idx': -1,
                            'pitch': message.note,
                            'time_on': mido.tick2second( cummulative_time, s.ticks_per_beat, tempo ),
                            'time_off': -1
                        }
                        note_ons_offs.append( tmp_note_obj )
                        note_on_num += 1
                    else:
                        if note_ons[message.note] > -1:
                            note_ons_offs[ note_ons[message.note] ]['off_idx'] = i_message
                            note_ons_offs[ note_ons[message.note] ]['time_off'] = mido.tick2second( cummulative_time, s.ticks_per_beat, tempo )
                            note_ons[message.note] = -1
                        else:
                            print('ERROR: note off without previous note one')
    return note_ons_offs
# get_noteOnOffs_from_midi_track
noteOnOffs = get_noteOnOffs_from_midi_track(song, info_dictionary, 'bass')
print(noteOnOffs)

[{'on_idx': 8, 'off_idx': 9, 'pitch': 34, 'time_on': 1.2, 'time_off': 2.3991666666666664}, {'on_idx': 10, 'off_idx': 11, 'pitch': 41, 'time_on': 2.4, 'time_off': 3.5991666666666666}, {'on_idx': 12, 'off_idx': 13, 'pitch': 44, 'time_on': 3.5999999999999996, 'time_off': 4.799166666666666}, {'on_idx': 14, 'off_idx': 15, 'pitch': 51, 'time_on': 4.8, 'time_off': 5.399166666666666}, {'on_idx': 16, 'off_idx': 17, 'pitch': 56, 'time_on': 5.3999999999999995, 'time_off': 5.9991666666666665}, {'on_idx': 18, 'off_idx': 19, 'pitch': 34, 'time_on': 6.0, 'time_off': 7.199166666666666}, {'on_idx': 20, 'off_idx': 21, 'pitch': 41, 'time_on': 7.199999999999999, 'time_off': 8.199166666666667}, {'on_idx': 22, 'off_idx': 23, 'pitch': 41, 'time_on': 8.2, 'time_off': 8.399166666666666}, {'on_idx': 24, 'off_idx': 25, 'pitch': 47, 'time_on': 8.399999999999999, 'time_off': 8.999166666666666}, {'on_idx': 26, 'off_idx': 27, 'pitch': 42, 'time_on': 9.0, 'time_off': 9.199166666666667}, {'on_idx': 28, 'off_idx': 29, 

In [34]:
# number of alterations
n = 10
# semitones available
s = [12, 7, 3, 6]
# select alterations at random
random_idxs = np.random.permutation(len(note_ons_offs))[:n]
# perform alterations
for i in random_idxs:
    note_ons_offs[i]['pitch'] += s[ np.random.randint( len(s) ) ]